# 量子信号处理与量子奇异值变换

*Copyright (c) 2022 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

## 概览

**量子信号处理**（quantum signal processing, QSP）是一个首次由 Guang Hao Low 和 Issac L. Chuang [[1]](https://quantum-journal.org/papers/q-2019-07-12-163/) 提出的量子算法，其使用反射（$R_X$）和旋转（$R_Z$）算子来模拟标量的多项式变换。对于满足特定条件的多项式（比如切比雪夫多项式），QSP 表明这些多项式变换可以通过单比特上的量子门模拟。使用名为“酉算子线性组合”（linear combination of unitaries, LCU）的技巧，我们只需增加两个额外辅助比特，就能模拟任意复多项式变换。简而言之，QSP可拟合任意能被泰勒级数逼近的函数。

论文 [[2]](https://doi.org/10.1145/3313276.3316366)进一步发展了 QSP 的思想，并提出使用高维旋转算子的量子算法来模拟矩阵的多项式变换。由于矩阵的多项式变换本质上是它的奇异值的多项式变换，因此该算法被称为**量子奇异值变换**（quantum singular value transformation, QSVT）。另外，我们还介绍名为**单比特化**（qubitization）的技术，该技术可使用一个辅助比特就能模拟高维旋转算子，大幅减少了模拟高维度旋转算子的资源消耗，使得我们能够用量子电路实现矩阵的多项式变换。

基于论文 [[2]](https://doi.org/10.1145/3313276.3316366)，该教程会向大家介绍 QSP、QSVT 和单比特化的概念及其在量桨上的实现。

接下来，我们先介绍块编码（block-encoding）的概念。

以下是必要的 libraries 和 packages。

In [17]:
import numpy as np
from numpy.polynomial.polynomial import Polynomial
from numpy.polynomial import Chebyshev
import paddle

# 量桨的通用函数
import paddle_quantum
from paddle_quantum.ansatz import Circuit
from paddle_quantum.qinfo import dagger
from paddle_quantum.linalg import unitary_random_with_hermitian_block, density_matrix_random

# 量桨的 QSVT 模块函数
from paddle_quantum.qsvt import poly_matrix, ScalarQSP, Phi_verification, reflection_based_quantum_signal_processing, QSVT

## 块编码

（量子）块编码是一种给矩阵数据编码的方式。在量子计算中，对于一个给定的矩阵 $A \in \mathbb{C}^{2^n \times 2^n}$，我们能找到一个酉矩阵 $U \in \mathbb{C}^{2^m \times 2^m}$ 满足如下等式

$$
W U V = 
\begin{bmatrix}
    A & 0 \\
    0 & 0
\end{bmatrix}
= A \oplus 0I^{\otimes (m - n)}, \tag{1}
$$

其中，$W, V \in \mathbb{C}^{2^m \times 2^m}$ 是两个正交投影算子，这样的酉矩阵 $U$ 被称为 $A$ 的**块编码**（block encoding）。

一般地，投影算子通常是 $W = V = |0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes I^{\otimes n}$，那么酉矩阵 $U$ 在标准正交基下可表示为

$$
U = \begin{bmatrix}
    A & ... \\
    ... & ...
\end{bmatrix}, \tag{2}
$$

即 $A$ 位于 $U$ 的左上角。

### 编码与解码

一般来说，块编码的表达形式并非是唯一的。比如，当 $A$ 可以对角化时，$U$ 可以被构建为

$$
U = \begin{bmatrix}
    A & i\sqrt{I^{\otimes n} - A^2} \\
    i\sqrt{I^{\otimes n} - A^2} & A
\end{bmatrix}; \tag{3}
$$

当 $A$ 是酉矩阵时，我们则可以选择 $U = A \oplus I^{\otimes (m - n)}$，即 $U$ 是受控的 $A$。特别地，文献[[3]](http://arxiv.org/abs/2203.10236) [[4]](http://arxiv.org/abs/2206.03505)给出了一些使用量子电路实现块编码的方案。

In [18]:
num_qubits = 3
num_block_qubits = 2

# 创建一个 2 比特大小的厄密矩阵 A 的 3 比特大小的块编码 U
# create a 3-qubit block encoding unitary U of a random 2-qubit Hermitian matrix A
U = unitary_random_with_hermitian_block(num_qubits)
A = U[0: 2 ** num_block_qubits, 0: 2 ** num_block_qubits]

另一方面，我们也可以从 矩阵 $A \in \mathbb{C}^{2^n \times 2^n}$ 的块编码 $U$ 中解码出 $A$ 的信息。用$|0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes \rho$表示输入量子态，块编码 $U$ 作为量子电路，那么输出量子态为

$$
U (|0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes \rho) U^\dagger
= \begin{bmatrix}
    A \rho A^\dagger & ... \\
    ... & ...
\end{bmatrix}
= |0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes A \rho A^\dagger + (I^{\otimes (m - n)} - |0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)}) \otimes ... \tag{4}
$$

对 $|0\rangle^{\otimes (m - n)}$ 所在的寄存器作测量，若结果为 $0^{\otimes (m - n)}$，那么我们成功地提取了 $A$ 的信息
<!-- 测量第一个量子寄存器。若结果为 $0^{\otimes (m - n)}$，则信息提取成功。 -->

![block-decoding](figures/QSVT-fig-block-decoding.png "图 1：块解码的量子实现")

解码的成功概率取决于测量到 $0^{\otimes (m - n)}$ 的几率，其会随着 $m - n$ 的增长而指数性下降。然而，若我们可以控制 $m - n$ 的大小，那么这就不再是一个问题。在本教程中我们会假设 $m = n + 1$.

In [19]:
# 设置密度矩阵后端
paddle_quantum.set_backend('density_matrix')

# 定义输入态
rho = density_matrix_random(num_block_qubits)
zero_state = paddle.eye(2 ** (num_qubits - num_block_qubits), 1) # 创建 0 态的矢量
input_state = paddle_quantum.State(paddle.kron(zero_state @ dagger(zero_state), rho))

# 定义辅助寄存器
aux_register = list(range(num_qubits - num_block_qubits))

# 创建线路
cir = Circuit(num_qubits)
cir.oracle(U, list(range(num_qubits)))
cir.collapse(aux_register, desired_result='0', if_print = True) # 调用塌缩算子

# 获取输出态及输出 rho
output_state = cir(input_state)
output_rho = output_state.data[0: 2 ** num_block_qubits, 0: 2 ** num_block_qubits]

qubits [0] collapse to the state |0> with probability 0.24548491835594177


我们可以验证 ``output_rho`` 与 $\frac{A \rho A^\dagger}{\text{Tr}(A \rho A^\dagger)}$ 相同。

In [20]:
expect_rho = A @ rho @ dagger(A)
expect_rho /= paddle.trace(expect_rho)
print(f"期望 rho 和 输出 rho 的误差是 {paddle.norm(paddle.abs(expect_rho - output_rho)).item()}")

期望 rho 和 输出 rho 的误差是 0.0


当 $m = 1$（因此 $n = 0$）时，$A$ 是一个标量。在这种情况下我们则可以算出 $U$ 对于量子态 $|0\rangle$ 的期望值来获取 $A$。

## 量子信号处理

论文 [[2]]((https://doi.org/10.1145/3313276.3316366)) 中的 Theorem 3-4 证明，若一个次数为 $k$ 的复数多项式 $P \in \mathbb{C}[x]$ 满足

- 若 $k$ 为偶/奇数，则 $P$ 是一个偶/奇多项式；
- 对于所有的 $x \in [-1, 1]$，$|P(x)| \leq 1$;
- 对于所有的 $x \in (-\infty, -1] \cup [1, \infty)$，$|P(x)| \geq 1$;
- 若 $k$ 为偶，则 $P(ix)P^*(ix) \geq 1$,

则存在多项式 $Q \in \mathbb{C}[x]$ 和角度矢量 $\Phi = (\phi_0, ..., \phi_k) \in \mathbb{R}^{k + 1}$ 使得对于所有的 $x \in [-1, 1]$,

$$
W_\Phi(x) := R_Z(-2\phi_0) \prod_{j = 1}^k R_X(\arccos(x)) R_Z(-2\phi_j)
= \begin{bmatrix}
   P(x) & i Q(x)\sqrt{1 - x^2} \\
   iQ^*(x)\sqrt{1 - x^2} & P^*(x) 
\end{bmatrix}. \tag{5}
$$

换句话说，我们可以使用 $k + 1$ 个旋转（$R_Z$）门和 $k$ 个反射 ($R_X$) 门去获得一个 $P(x)$ 的块编码酉矩阵 $W_\Phi(x)$。块解码后 $P(x)$ 的模拟就完成了。

以下是一些满足上述条件的例子。

In [21]:
# 简单的奇多项式
# P = Polynomial([0, 0, 0, 0, 0, 1])
# P = Polynomial([0, 0.5, 0, 0, 0, 0.5])
P = Polynomial([0, 1 / 3, 0, 0, 0, 1 / 3, 0, 1 / 3])

# 次数为10的第一类切比雪夫多项式
# P = Chebyshev([0 for _ in range(10)] + [1]).convert(kind=Polynomial)

# 次数为11的第一类切比雪夫多项式
# P = Chebyshev([0 for _ in range(11)] + [1]).convert(kind=Polynomial)

值得注意的是，$W_\Phi$ 可以模拟切比雪夫多项式变换。对于次数为 $k$ 的第一类切比雪夫多项式，其对应的角度矢量为 $(0, \pi, ..., \pi)$ （如 $k$ 为偶数）或 $(\pi, ..., \pi)$ (如 $k$ 为奇数).

量桨的 QSVT 模块有一个内置类 `ScalarQSP` 用于完成量子信号处理，并可以调用函数 `Phi_verification` 来验证是否 $W_\Phi$ 是 $P$ 的块编码。

In [22]:
qsp = ScalarQSP(P)
print(qsp.Phi)
assert Phi_verification(qsp.Phi.numpy(), P)

Tensor(shape=[8], dtype=float64, place=Place(cpu), stop_gradient=True,
       [ 3.14159265,  1.39460474, -0.44313783,  1.09757975, -1.09757975,
         0.44313783, -1.39460474,  3.14159265])


我们也可以通过量子线路来验证 $\Phi$ 的正确性。

In [23]:
x = np.random.rand() * 2 - 1 # 随机在 [-1, 1] 间选取 x
cir = qsp.block_encoding(x)
print(cir)
print(f"模拟 P(x) 的误差是 {np.abs(cir.unitary_matrix()[0, 0].item() - P(x))}")

--Rz(-6.28)----Rx(-2.73)----Rz(2.789)----Rx(-2.73)----Rz(-0.88)----Rx(-2.73)----Rz(2.195)----Rx(-2.73)----Rz(-2.19)----Rx(-2.73)----Rz(0.886)----Rx(-2.73)----Rz(-2.78)----Rx(-2.73)----Rz(-6.28)--
                                                                                                                                                                                                   
模拟 P(x) 的误差是 1.7893723625681406e-08


### QSP 变种

另外，参数数量可以被进一步减少至 $k$，通过找到一个角度矢量 $\Phi \in \mathbb{R}^k$ 使得

$$
R_\Phi(x):=\prod_{i = 1}^{k} e^{i \phi_i \sigma_z} R(x)
= \begin{bmatrix}
    P(x) & ... \\
    ... & ...
\end{bmatrix}, \tag{6}
$$

这里

$$
R(x) = \begin{bmatrix}
    x & \sqrt{1 - x^2} \\
    \sqrt{1 - x^2} & -x
\end{bmatrix}. \tag{7}
$$

在量桨里我们通过调用 `reflection_based_quantum_signal_processing` 来计算 $\Phi$。

In [24]:
print(reflection_based_quantum_signal_processing(P))

[15.70796327 -0.17619159 -2.01393416 -0.47321657 -2.66837608 -1.12765849
 -2.96540107]


> 注: $R_{-\Phi}(x)$ 是 $P^*(x)$ 的块编码。

## 量子奇异值变换

接下来，我们介绍量子奇异值变换的概念。

对于 $\mathcal{X} \in \mathbb{C}^{2^m \times 2^m}$ 为一个矩阵，当它可以被一个酉矩阵 $U$ 和两个正交投影算子 $ W, V$ 表示时，即 $\mathcal{X} = W U V$，那么矩阵 $\mathcal{X}$ 有如下表达式：

$$
\mathcal{X} = \sum_{j=1}^{2^m} \lambda_j |\omega_j\rangle \langle\nu_j|, \tag{8}
$$

其中 $\{|\omega_j\rangle\}_{j=1}^{2^m}$ 和 $\{|\nu_j\rangle\}_{j=1}^{2^m}$ 是由 $W$ 和 $V$ 的本征态扩展生成的标准正交基，且 $\lambda_j$ 是 $\mathcal{X}$ 的奇异值，

$$
\lambda_j = \begin{cases}
\langle\omega_j| U |\nu_j\rangle \text{ 如 } j \leq \text{rank}(\mathcal{X}) \\
0 \text{  其他情况 }
\end{cases} \tag{9}
$$

对于给定的周期性函数 $f$，矩阵$\mathcal{X}$ 的**奇异值变换**(SVT) 定义为

$$
f^{(SV)}(\mathcal{X}) := \sum_{j=1}^{2^m} f(\lambda_j)
\begin{cases}
       |\nu_j\rangle\langle\nu_j| & \text{  如 } f \text{ 是偶函数} \\
       |\omega_j\rangle\langle\nu_j| & \text{  如 } f \text{ 是奇函数}
\end{cases}. \tag{10}
$$

### 分解与 QSP

论文 [[2]]((https://doi.org/10.1145/3313276.3316366)) 证明在**合适**的基下, 与 $\lambda_j$ 相关的子空间可以将 $U, V$ 和 $W$ 分解为

$$
\begin{split}
U & = \bigoplus_{j: \lambda_j \in [0, 1)} R(\lambda_j) \oplus
        \bigoplus_{j: \lambda_j = 1} [1] \oplus
        \bigoplus_{j: V|\nu_j\rangle = |\nu_j\rangle,\, WU|\nu_j\rangle = 0} [1] \oplus
        \bigoplus_{j: W|\omega_j\rangle = |\omega_j\rangle,\, VU^\dagger|\omega_j\rangle = 0} [1] \oplus 
        \bigoplus_{j: V|\nu_j\rangle = W|\omega_j\rangle = 0} [...], \\
V & = \bigoplus_{j: \lambda_j \in [0, 1)} |0\rangle\langle0| \oplus
        \bigoplus_{j: \lambda_j = 1} [1] \oplus
        \bigoplus_{j: V|\nu_j\rangle = |\nu_j\rangle,\, WU|\nu_j\rangle = 0} [1] \oplus
        \bigoplus_{j: W|\omega_j\rangle = |\omega_j\rangle,\, VU^\dagger|\omega_j\rangle = 0} [0]  \oplus 
        \bigoplus_{j: V|\nu_j\rangle = W|\omega_j\rangle = 0} [0] \text{ and}\\
W & = \bigoplus_{j: \lambda_j \in [0, 1)} |0\rangle\langle0| \oplus
        \bigoplus_{j: \lambda_j = 1} [1] \oplus
        \bigoplus_{j: V|\nu_j\rangle = |\nu_j\rangle,\, WU|\nu_j\rangle = 0} [0] \oplus
        \bigoplus_{j: W|\omega_j\rangle = |\omega_j\rangle,\, VU^\dagger|\omega_j\rangle = 0} [1]  \oplus 
        \bigoplus_{j: V|\nu_j\rangle = W|\omega_j\rangle = 0} [0] .\\
\end{split} \tag{11}
$$

$f^{(SV)}$ 也可以被分解为

$$
f^{(SV)}(\mathcal{X}) = \sum_{j: \lambda_j \in [0, 1)} f(\lambda_j)\,... + \sum_{j: \lambda_j = 1} f(1)\,... + \sum_{j: \lambda_j = 0, V |\nu_j\rangle \neq 0} f(0)\,... + \sum_{j: \lambda_j = 0, W |\omega_j\rangle \neq 0} f(0)\,... + \sum_{j: \lambda_j = 0, \text{otherwise}} f(0)\,...\,, \tag{12}
$$


现在我们将上述分解与量子信号处理联系到一起。

假设函数 $P$ 是一个次数为 $k$ 并满足上节所述条件，则存在 $\Phi \in \mathbb{R}^k$ 使得 $R_\Phi$ 是 $P$ 的块编码，并且 $P$ 满足 $P(1) = e^{i \sum_{j=1}^k \phi_j}$ 和 $P(0) = e^{i \sum_{j=1}^k (-1)^{j+1} \phi_j}$。

定义 $U_\Phi$ 为

$$
U_\Phi :=  \begin{cases}
& \prod_{j = 1}^{k / 2} e^{i\phi_{2j - 1} (2V - I)} U^\dagger e^{i\phi_{2j} (2W - I)} U \text{  如 } k \text{ 是偶数} \\
e^{i\phi_1 (2W - I)} U & \prod_{j = 1}^{(k - 1) / 2} e^{i\phi_{2j} (2V - I)} U^\dagger e^{i\phi_{2j+1} (2W - I)} U \text{  如 } k \text{ 是奇数}
\end{cases}. \tag{13}
$$

那么

$$
U_\Phi =  \begin{cases}
        \bigoplus R_\Phi(\lambda_j) \oplus
        \bigoplus [e^{i \sum_{j=1}^k \phi_j}] \oplus
        \bigoplus [e^{i \sum_{j=1}^k (-1)^{j+1} \phi_j}] \oplus
        \bigoplus [e^{i \sum_{j=1}^k (-1)^{j+1} \phi_j}] \oplus 
        \bigoplus [...]
\text{  如 } k \text{ 是偶数} \\
        \bigoplus R_\Phi(\lambda_j) \oplus
        \bigoplus [e^{i \sum_{j=1}^k \phi_j}] \oplus
        \bigoplus [e^{i \sum_{j=1}^k (-1)^{j+1} \phi_j}] \oplus
        \bigoplus [e^{i \sum_{j=1}^k (-1)^{j+1} \phi_j}] \oplus 
        \bigoplus [...]
\text{  如 } k \text{ 是奇数}
\end{cases}, \tag{14}
$$

因此

$$
P^{(SV)}(\mathcal{X}) = \begin{cases} 
        \bigoplus \begin{bmatrix}
                P(\lambda_j) & 0 \\
                0 & 0 \\
        \end{bmatrix} \oplus
        \bigoplus [P(1)] \oplus
        \bigoplus [P(0)] \oplus
        \bigoplus [0] \oplus 
        \bigoplus [0] = V U_\Phi V
\text{  如 } k \text{ 是偶数} \\ 
        \bigoplus \begin{bmatrix}
                P(\lambda_j) & 0 \\
                0 & 0 \\
        \end{bmatrix} \oplus
        \bigoplus [P(1)] \oplus
        \bigoplus [0] \oplus
        \bigoplus [0] \oplus 
        \bigoplus [0] = W U_\Phi V 
\text{  如 } k \text{ 是奇数}
\end{cases}. \tag{15}
$$

若是我们可以使用量子算法实现 $V U_\Phi V$ 或者 $W U_\Phi V$，那么这样的变换就是 $\mathcal{X}$ 的量子奇异值变换。

### QSVT 和块编码

假设相对于正交投影算子 $W$ 和 $V$， $U$ 是一个 $A$ 的块编码酉矩阵。那么 $\mathcal{X} = A \oplus 0I^{\otimes (m - n)}$ 且 $P^{(SV)}(\mathcal{X}) = P^{(SV)}(A) \oplus 0I^{\otimes (m - n)}$. 因此，$U_\Phi$ 是 $P^{(SV)}(A)$ 的块编码.

当 $W = V$ 时，$\{|\omega_j\rangle \}_{j=1}^{2^m} = \{ |\nu_j\rangle \}_{j=1}^{2^m}$。记 $P(x) := \sum_{i=0}^k c_i x^i$，我们就可以找到

$$
P^{(SV)}(\mathcal{X}) = \sum_{j=1}^{2^m} P(\lambda_j) |\nu_j\rangle \langle\nu_j| = P(X) = P(A) \oplus 0I^{\otimes (m - n)}. \tag{16}
$$

换句话说，当 $W = V$ 时，QSVT 将一个 $A$ 的块编码转为了一个 $P(A)$ 的块编码。

当 $W = V = |0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes I^{\otimes n}$ 时，量桨有一个内置类 `QSVT` 来完成量子奇异值变换。我们可以调用其成员函数 `QSVT.block_encoding_matrix()` 来验证上述理论的正确性。

In [25]:
qsvt = QSVT(P, U, num_block_qubits)

In [26]:
# 找到 P(A) 及
# find P(A) and its expected eigenvalues, note that they are computed in different ways
expect_PA = poly_matrix(P, A).numpy()
expect_eig_result = np.sort(list(map(lambda x: P(x), np.linalg.eigvals(A.numpy()))))

# Calculate U_\Phi and extract eigenvalues of block encoded matrix
U_Phi = qsvt.block_encoding_matrix().numpy()
actual_PA = U_Phi[0:2 ** num_block_qubits, 0:2 ** num_block_qubits]
actual_eig_result = np.sort(np.linalg.eigvals(actual_PA))

In [27]:
print("模拟 P(X) 的误差")
print("     最大绝对值, ", np.amax(abs(expect_PA - actual_PA)))
print("     百分比, ", np.linalg.norm(expect_PA - actual_PA) / np.linalg.norm(expect_PA))

模拟 P(X) 的误差
     最大绝对值,  1.450928305889582e-07
     百分比,  3.370838085602552e-07


In [28]:
print("模拟 P(X) 的本征值误差")
print("     最大绝对值, ", np.amax(abs(expect_eig_result - actual_eig_result)))
print("     百分比, ", np.linalg.norm(expect_eig_result - actual_eig_result) / np.linalg.norm(expect_eig_result))

模拟 P(X) 的本征值误差
     最大绝对值,  1.8375562631798232e-07
     百分比,  2.776073800450616e-07


### 单比特化: $U_\Phi$ 的量子实现

现在的问题是，如何用量子线路去实现 $U_\Phi$ ？通常我们认为块编码 $U$ 已经通过量子电路实现，那么剩下来只需实现 $e^{i\phi (2W - I)}$ 和 $e^{i\phi (2V - I)}$。值得注意的是，这两个算子本质上是分别在 $W$ 和 $V$ 的投影空间下施加的 $R_Z(-2\phi)$ 算子。

论文 [[1]](https://quantum-journal.org/papers/q-2019-07-12-163/) 中的 Lemma 10 认为，应该将投影算子的映射空间投射到一个辅助比特上再进行旋转，那么通过纠缠的性质这个旋转也会同时应用于主寄存器上。

这些理论被总结至下图：

![U_Phi](figures/QSVT-fig-U_Phi.png "图 2: QSVT 的量子实现，这里 k 是 P 的多项式次数")

当 $W = V = |0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes I^{\otimes n}$ 时，量桨可以调用成员函数 `QSVT.block_encoding_circuit` 来创建一个图 2 所示的量子线路。我们可以通过比较该线路的输出态和量子态 $(U_\Phi \otimes I)|\psi\rangle|0\rangle$（$|\psi\rangle \in \mathbb{C}^{2^m}$）来证明所建立的线路是可以模拟 $U_\Phi$ 的。

In [29]:
# 设置态矢量后端
paddle_quantum.set_backend('state_vector')

# 随即量子态满足最后一个量子比特固定为 0 态
ket_0 = [1.0, 0.0]
psi = np.array([np.random.rand() + 1j * np.random.rand() for _ in range(2 ** num_qubits)])
psi = np.kron(psi / np.linalg.norm(psi), ket_0)

In [30]:
expect_state = np.kron(U_Phi, np.eye(2)) @ psi

cir = qsvt.block_encoding_circuit()
actual_state = cir(paddle_quantum.State(psi, dtype='complex64')).data.numpy()

print(f"期望量子态和实际量子态的差距是 {np.linalg.norm(expect_state - actual_state)}")

期望量子态和实际量子态的差距是 3.0184617834444857e-07


> 注：如果我们在图 2 中的辅助寄存器两边加上 Hadamard 门，那么该量子线路就会转为模拟其多项式的实数变化。再结合 LCU 的技巧，理论上我们只需要额外 1 个量子比特来量子模拟任意范数小于 1 的复数多项式。

## 应用: 振幅放大

设 $|\psi\rangle$ 为 $m$ 比特大小的量子态并满足 $|\psi\rangle = \sin(\theta)|\psi_{\text{good}}\rangle + \cos(\theta) |\psi_{\text{bad}}\rangle$。**振幅放大**是一个用于增大 $|\psi_{\text{good}}\rangle$ 至 1 的量子算法。该算法可以从另一个角度来解释。设定 $U$ 为一个酉矩阵且 $W$ 和 $V$ 为两个正交投影算子，使得 $|\psi\rangle = U V |0\rangle^{\otimes m}$ 且 $\sin(\theta)|\psi_{\text{good}}\rangle = W |\psi\rangle$，意味着 $\sin(\theta)|\psi_{\text{good}}\rangle = W U V |0\rangle^{\otimes m}$。因此，振幅放大本质上是 $\mathcal{X} = W U V$ 的奇异值变换。
在这一章节我们会根据论文 [[2]]((https://doi.org/10.1145/3313276.3316366)) 的 Theorem 28，来展示如何用 QSVT 来做定点（即 $\theta$ 满足 $\theta = \frac{\pi}{2k}$，$k \in \mathbb{Z}$）振幅放大算法。

假设 $\sin(\theta)|\psi_{\text{good}}\rangle = (|0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes I^{\otimes n}) |\psi\rangle$ 且 $\theta = \frac{\pi}{2k}$（$k \in \mathbb{Z}$）。那么就能推导出 $W = V = |0\rangle^{\otimes (m - n)}\langle0|^{\otimes (m - n)} \otimes I^{\otimes n}$ 且 $\mathcal{X} = A \oplus 0I^{\otimes (m - n)}$，这里 $A$ 位于 $U$ 的左上角。 

可见 $\mathcal{X} |0\rangle^{\otimes m} = \sin(\frac{\pi}{2k}) |\psi_\text{good}\rangle$。因此，我们旨在找到一个酉矩阵为 $U_\Phi$ 的量子线路使得 $W U_\Phi V = \sin^{-1}(\theta) \mathcal{X}$， 从而达到 $W U_\Phi V |0\rangle^{\otimes m} = |\psi_{\text{good}}\rangle$ 的目的。因为 $\mathcal{X} = W U V$，$\mathcal{X}$ 的奇异值的绝对值是 $\sin(\frac{\pi}{2k})$。我们进一步选择 $P(x) = (-1)^k T_k (x)$，这里 $T_k$ 是次数为 $k$ 的第一类切比雪夫多项式。最后通过

$$
P(\sin(\frac{\pi}{2k})) = (-1)^k T_k (\sin(\frac{\pi}{2k})) = (-1)^k \cos(\frac{k - 1}{2}\pi) = 1, \tag{17}
$$

我们可得，对于多项式 $P$ 的 $\mathcal{X}$ 的量子奇异值变换是 $B := \frac{1}{\sin(\frac{\pi}{2k})} A$ 的块编码。

设定 $k = 3$ 使得 $\sin(\frac{\pi}{2k}) = \frac{1}{2}$，同时随机选择酉矩阵 $U$。其左上角为 $A$，我们需要证明 $U$ 的 QSVT 的左上角为 $B = 2A$。

In [31]:
P = -1 * Chebyshev([0 for _ in range(3)] + [1]).convert(kind=Polynomial)
U = unitary_random_with_hermitian_block(num_qubits, is_unitary=True)
A = U[0:2 ** num_block_qubits, 0:2 ** num_block_qubits].numpy()

amplifier = QSVT(P, U, num_block_qubits)
U_Phi = amplifier.block_encoding_unitary()

In [32]:
B = U_Phi[0:2 ** num_block_qubits, 0:2 ** num_block_qubits].numpy()
print(f"QSVT 的精确值为 {np.linalg.norm(B - 2 * A)}")

QSVT 的精确值为 3.1799856969882967e-07


由上可见，我们成功将 $A$ 的大小翻倍。

___
## 参考文献

[1] Low, Guang Hao, and Isaac L. Chuang. "Hamiltonian simulation by qubitization." [Quantum 3 (2019): 163.](https://doi.org/10.22331/q-2019-07-12-163)

[2] Gilyén, András, et al. "Quantum singular value transformation and beyond: exponential improvements for quantum matrix arithmetics." [Proceedings of the 51st Annual ACM SIGACT Symposium on Theory of Computing. 2019.](https://doi.org/10.1145/3313276.3316366)

[3] Camps, Daan, et al. "Explicit Quantum Circuits for Block Encodings of Certain Sparse Matrices." [arXiv preprint arXiv:2203.10236 (2022).](http://arxiv.org/abs/2203.10236)

[4] Clader, B. David, et al. "Quantum Resources Required to Block-Encode a Matrix of Classical Data." [arXiv preprint arXiv:2206.03505 (2022).](http://arxiv.org/abs/2206.03505)
